In [1]:
import pandas as pd
import numpy as np
import mlflow
import json
import re
import uuid

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold


In [2]:
# 데이터 전처리 코드 기존 데이터를 읽어와 전처리된 csv 파일로 저장(또는 곧바로 사용 가능)
# 데이터 읽어오기 및 전처리 함수
def parse_json(item):
    try:
        data = json.loads(item)  # JSON 문자열을 파싱
        if isinstance(data, dict):  # 단일 딕셔너리라면 리스트로 감싸기
            return [data]
        return data  # 리스트라면 그대로 반환
    except json.JSONDecodeError:
        return None  # JSON 파싱 오류 발생 시 None 반환

def json_column_to_column(df):
    df['parsed_data'] = df['contents'].apply(parse_json)
    df_exploded = df.explode('parsed_data', ignore_index=True)
    return pd.concat([df_exploded.drop(columns=['contents', 'parsed_data']), pd.json_normalize(df_exploded['parsed_data'])], axis=1)

# 데이터 읽어오기
df = pd.read_csv('./take_hotel.csv')
convert_df = json_column_to_column(df)
convert_df = convert_df[['room_name', 'reg_date', 'itemName']]

# 키워드 리스트로 필터링
keywords = [
    "Chicken Karaage & Fried  Dishes (치킨 가라아게 & 모둠 튀김)", "Beef Truffle Zappaghetti(소고기 트러플 짜파게티)",
    "Mini Burger Set (미니버거 세트) 음료 선택 필수", "Bluemoon Draft Beer (블루문 생맥주)", "Miller Draft Beer (밀러 생맥주)",
    "Pig Hocks Set (순살 족발 세트)", "Beef Brisket Kimchi Pilaf(차돌 깍두기 볶음밥)", "Zero Coke (제로 콜라)",
    "Margherita Pizza(마르게리따 피자)", "Shine Musket Ade (샤인머스켓 에이드)", "Hamburg Steak (함박 스테이크)",
    "Truffle mushrooms Pizza(트러플 버섯 피자)", "Tteokbokki(떡볶이)", "Truffle Creamy Gnocchi Pasta(트러플 크림 뇨끼)",
    "Americano (아메리카노) 요청사항에 HOT/ICE 부탁드립니다.", "Lemon Ade (레몬 에이드)", "Coke (코카콜라)",
    "Seafood Oil Casareccia Pasta (씨푸드 오일 파스타)", "Prosciutto Rucola Pizza (프로슈토 루꼴라 피자)", 
    "Abalone Porridge + Side  Dishes (전복죽 + 반찬)", "Starlight Chungha (별빛 청하)", "LA Galbi Set (LA 갈비 세트)", 
    "Zero Sprite(제로 스프라이트)", "Grilled Camembert Cheese(까망베르 치즈 구이)", "Grapefruit Ade (자몽 에이드)",
    "Pancake Plate(팬케이크 플레이트)", "Sprite (스프라이트)", "Chicken Burger(치킨버거)", "Margherita Pizza(마르게리따타피자)",
    "Prosciutto Rucola Pizza(프로슈토 루꼴라 피자)", "Americano (아메리카노)", "Hwayo25 Soju (화요 25)", 
    "Mini Burger set", "Korea Traditional Pancakes (모둠 전)", "ILPUM Jinro Soju (일품 진로 소주)",
    "Abalone Porridge  (전복죽)", "Mini Burger Set (미니버거 세트)", "Creamy Pasta (크림 파스타)", 
    "Melon prosciutto(메론 프로슈토)", "Shine musket(샤인머스켓 에이드)", "Abalone Porridge + side (전복죽 + 반찬)",
    "Chicken Karaage & Fries Plate(치킨 가라아게 & 모둠튀김)", "Seafood Oil Casareccia Pasta(씨푸드 오일 까사레치아 파스타)", 
    "miller Draft beer  (밀러 맥주)", "Pig Hocks set(순살 족발세트)", "Beer (캘리)", "Coca cola(콜라)", 
    "Melon Prosciutto (메론 프로슈토)", "Seafood Oil Casareccia Pasta(씨푸트 오일 까사레치아 파스타)", 
    "LA Galbi set(LA 갈비 세트)", "bluemoon Draft beer  (블루문 맥주)", "Hamburg Steak(함박 스테이크)", 
    "Sprite(스프라이트)", "Soju (새로)", "Scones(스콘)", "Coca cola(코카콜라)", "Sprite(스프라이트, 제로 택 1)",
    "Korea Traditional Pancake(모둠전)", "ILpum Jinro Soju (일품 진로 소주)", "Zero cola (제로 콜라)", 
    "Chicken Karaage & Fried  Plate", "Seafood Oil Casareccia Pasta", "Starlight Chung Ha(달빛 청하)", 
    "아메리카노", "Hwayo25 soju(화요 25 소주)", "Grapefruit Ade(자몽 에이드)", "Draft beer(bluemoon, 생맥주)", 
    "Prosciutto Rucola Pizza", "LA Galbi set", "Pig Hocks set (순살 족발 세트)", "Draft beer(bluemoon,  miller 택 1)"
]


escaped_keywords = [re.escape(keyword) for keyword in keywords]
pattern = '|'.join(escaped_keywords)  # 이스케이프된 키워드를 '|'로 연결

filtered_df = convert_df[convert_df['itemName'].str.contains(pattern, case=False, na=False)]

# 키워드 매핑
keyword_dict = {
    # Chicken and Fried Dishes
    "Chicken Karaage & Fried  Dishes (치킨 가라아게 & 모둠 튀김)": "Chicken Karaage & Fried Dishes",
    "Chicken Karaage & Fries Plate(치킨 가라아게 & 모둠튀김)": "Chicken Karaage & Fried Dishes",
    "Chicken Karaage & Fried  Plate": "Chicken Karaage & Fried Dishes",

    # Beef and Other Main Dishes
    "Beef Truffle Zappaghetti(소고기 트러플 짜파게티)": "Beef Truffle Zappaghetti",
    "Beef Brisket Kimchi Pilaf(차돌 깍두기 볶음밥)": "Beef Brisket Kimchi Pilaf",
    "LA Galbi Set (LA 갈비 세트)": "LA Galbi Set",
    "LA Galbi set(LA 갈비 세트)": "LA Galbi Set",
    "LA Galbi set": "LA Galbi Set",
    
    # Pig Hocks
    "Pig Hocks Set (순살 족발 세트)": "Pig Hocks Set",
    "Pig Hocks set(순살 족발세트)": "Pig Hocks Set",
    "Pig Hocks set (순살 족발 세트)": "Pig Hocks Set",
    
    # Burger
    "Mini Burger Set (미니버거 세트) 음료 선택 필수": "Mini Burger Set",
    "Mini Burger set": "Mini Burger Set",
    "Mini Burger Set (미니버거 세트)": "Mini Burger Set",
    "Chicken Burger(치킨버거)": "Chicken Burger",
    
    # Pizza
    "Margherita Pizza(마르게리따 피자)": "Margherita Pizza",
    "Margherita Pizza(마르게리따타피자)": "Margherita Pizza",
    "Prosciutto Rucola Pizza (프로슈토 루꼴라 피자)": "Prosciutto Rucola Pizza",
    "Prosciutto Rucola Pizza(프로슈토 루꼴라 피자)": "Prosciutto Rucola Pizza",
    "Prosciutto Rucola Pizza": "Prosciutto Rucola Pizza",
    "Truffle mushrooms Pizza(트러플 버섯 피자)": "Truffle Mushrooms Pizza",

    # Pasta and Gnocchi
    "Seafood Oil Casareccia Pasta (씨푸드 오일 파스타)": "Seafood Oil Casareccia Pasta",
    "Seafood Oil Casareccia Pasta(씨푸드 오일 까사레치아 파스타)": "Seafood Oil Casareccia Pasta",
    "Seafood Oil Casareccia Pasta(씨푸트 오일 까사레치아 파스타)": "Seafood Oil Casareccia Pasta",
    "Seafood Oil Casareccia Pasta": "Seafood Oil Casareccia Pasta",
    "Truffle Creamy Gnocchi Pasta(트러플 크림 뇨끼)": "Creamy Pasta",
    "Creamy Pasta (크림 파스타)": "Creamy Pasta",

    # Porridge
    "Abalone Porridge + Side  Dishes (전복죽 + 반찬)": "Abalone Porridge",
    "Abalone Porridge + side (전복죽 + 반찬)": "Abalone Porridge",
    "Abalone Porridge  (전복죽)": "Abalone Porridge",

    # Pancakes
    "Pancake Plate(팬케이크 플레이트)": "Korean Traditional Pancakes",
    "Korea Traditional Pancakes (모둠 전)": "Korean Traditional Pancakes",
    "Korea Traditional Pancake(모둠전)": "Korean Traditional Pancakes",

    # Others
    "Melon prosciutto(메론 프로슈토)": "Melon Prosciutto",
    "Melon Prosciutto (메론 프로슈토)": "Melon Prosciutto",
    "Hamburg Steak (함박 스테이크)": "Hamburg Steak",
    "Hamburg Steak(함박 스테이크)": "Hamburg Steak",
    "Tteokbokki(떡볶이)": "Tteokbokki",
    "Grilled Camembert Cheese(까망베르 치즈 구이)": "Grilled Camembert Cheese",
    "Scones(스콘)": "Scones",

    # Soju
    "Hwayo25 Soju (화요 25)": "Soju",
    "Hwayo25 soju(화요 25 소주)": "Soju",
    "ILPUM Jinro Soju (일품 진로 소주)": "Soju",
    "ILpum Jinro Soju (일품 진로 소주)": "Soju",
    "Soju (새로)": "Soju",
    "Starlight Chungha (별빛 청하)": "Soju",
    "Starlight Chung Ha(달빛 청하)": "Soju",

    # Beer
    "Bluemoon Draft Beer (블루문 생맥주)": "Beer",
    "Draft beer(bluemoon, 생맥주)": "Beer",
    "Draft beer(bluemoon,  miller 택 1)": "Beer",
    "bluemoon Draft beer  (블루문 맥주)": "Beer",
    "Miller Draft Beer (밀러 생맥주)": "Beer",
    "miller Draft beer  (밀러 맥주)": "Beer",
    "Beer (캘리)": "Beer",
    "Kelly Beer": "Beer",

    # 탄산음료
    "Coke (코카콜라)": "탄산음료",
    "Coca cola(콜라)": "탄산음료",
    "Coca cola(코카콜라)": "탄산음료",
    "Zero Coke (제로 콜라)": "탄산음료",
    "Zero cola (제로 콜라)": "탄산음료",
    "Sprite (스프라이트)": "탄산음료",
    "Sprite(스프라이트)": "탄산음료",
    "Sprite(스프라이트, 제로 택 1)": "탄산음료",
    "Zero Sprite(제로 스프라이트)": "탄산음료",

    # 에이드류
    "Lemon Ade (레몬 에이드)": "에이드",
    "Grapefruit Ade (자몽 에이드)": "에이드",
    "Grapefruit Ade(자몽 에이드)": "에이드",
    "Shine Musket Ade (샤인머스켓 에이드)": "에이드",
    "Shine musket(샤인머스켓 에이드)": "에이드",

    # Coffee and Tea
    "Americano (아메리카노) 요청사항에 HOT/ICE 부탁드립니다.": "Americano",
    "Americano (아메리카노)": "Americano",
    "아메리카노": "Americano"
}


filtered_df.loc[:, 'itemName'] = filtered_df['itemName'].replace(keyword_dict)
filtered_df

,room_name,reg_date,itemName
1556,910,2023-04-01 13:55:28,Mini Burger Set
1621,905,2023-04-02 12:29:13,Mini Burger Set
1622,905,2023-04-02 14:23:52,탄산음료
1623,905,2023-04-02 14:24:26,탄산음료
1656,907,2023-04-04 14:35:54,탄산음료
...,...,...,...
21718,2504,2024-10-25 22:13:45,Margherita Pizza
21719,2504,2024-10-25 22:13:45,탄산음료
21721,1901,2024-10-25 22:30:18,Beef Brisket Kimchi Pilaf
21722,1901,2024-10-25 22:30:18,에이드


In [ ]:
import requests
import json
from pprint import pprint

url = "http://<publicIP>:5001/invocations"
headers = {"Content-Type": "application/json"}

data = {
    "dataframe_split": {
        "columns": ["room_name"],
        "data": [2606,1906]
    }
}

response = requests.post(url, headers=headers, data=json.dumps(data))
pprint(response.json())

{'predictions': {'1906': ['Beer',
                          'Chicken Karaage & Fried Dishes',
                          '탄산음료',
                          'Mini Burger Set',
                          'Pig Hocks Set',
                          '에이드',
                          'Beef Truffle Zappaghetti',
                          'Soju',
                          'Americano',
                          'Abalone Porridge',
                          'Margherita Pizza',
                          'Prosciutto Rucola Pizza',
                          'Chicken Burger',
                          'Tteokbokki',
                          'LA Galbi Set',
                          'Seafood Oil Casareccia Pasta',
                          'Hamburg Steak',
                          'Beef Brisket Kimchi Pilaf',
                          'Creamy Pasta',
                          'Truffle Mushrooms Pizza',
                          'Melon Prosciutto',
                          'Korean Traditional Pancakes',

In [3]:
# csv파일저장
filtered_df.to_csv("filtered_take_hotel.csv",index=False)

In [4]:
# csv파일읽기
filtered_df = pd.read_csv("filtered_take_hotel.csv")
filtered_df

,room_name,reg_date,itemName
0,910,2023-04-01 13:55:28,Mini Burger Set
1,905,2023-04-02 12:29:13,Mini Burger Set
2,905,2023-04-02 14:23:52,탄산음료
3,905,2023-04-02 14:24:26,탄산음료
4,907,2023-04-04 14:35:54,탄산음료
...,...,...,...
3088,2504,2024-10-25 22:13:45,Margherita Pizza
3089,2504,2024-10-25 22:13:45,탄산음료
3090,1901,2024-10-25 22:30:18,Beef Brisket Kimchi Pilaf
3091,1901,2024-10-25 22:30:18,에이드


In [42]:
# train / test split
test_percent = 0.2
total_len = len(filtered_df)
train_len = int(total_len*(1-test_percent))

train_df = filtered_df[:train_len].copy()
test_df = filtered_df[train_len:].copy()

In [43]:
train_df.to_csv("train_original.csv",index=False)
test_df.to_csv("test_original.csv",index=False)

In [6]:
train_df

,room_name,reg_date,itemName
0,910,2023-04-01 13:55:28,Mini Burger Set
1,905,2023-04-02 12:29:13,Mini Burger Set
2,905,2023-04-02 14:23:52,탄산음료
3,905,2023-04-02 14:24:26,탄산음료
4,907,2023-04-04 14:35:54,탄산음료
...,...,...,...
2469,1405,2024-07-31 23:39:03,Beef Truffle Zappaghetti
2470,1405,2024-07-31 23:39:03,Pig Hocks Set
2471,1405,2024-07-31 23:39:03,Beer
2472,2310,2024-08-01 19:38:43,Mini Burger Set


In [7]:
test_df

,room_name,reg_date,itemName
2474,2306,2024-08-01 21:29:06,Beef Brisket Kimchi Pilaf
2475,2306,2024-08-01 21:29:06,Beef Truffle Zappaghetti
2476,2507,2024-08-01 21:55:27,Grilled Camembert Cheese
2477,1805,2024-08-01 21:58:07,Chicken Karaage & Fried Dishes
2478,1711,2024-08-01 22:23:19,Beer
...,...,...,...
3088,2504,2024-10-25 22:13:45,Margherita Pizza
3089,2504,2024-10-25 22:13:45,탄산음료
3090,1901,2024-10-25 22:30:18,Beef Brisket Kimchi Pilaf
3091,1901,2024-10-25 22:30:18,에이드


In [8]:
train_df['itemName'].value_counts()

itemName
Beer                              339
Chicken Karaage & Fried Dishes    294
Mini Burger Set                   223
탄산음료                              217
Beef Truffle Zappaghetti          178
에이드                               169
Pig Hocks Set                     145
Margherita Pizza                   91
Beef Brisket Kimchi Pilaf          87
Soju                               77
Americano                          73
Prosciutto Rucola Pizza            71
Hamburg Steak                      71
Seafood Oil Casareccia Pasta       69
Abalone Porridge                   67
Truffle Mushrooms Pizza            60
Tteokbokki                         52
Creamy Pasta                       51
LA Galbi Set                       50
Korean Traditional Pancakes        36
Grilled Camembert Cheese           26
Chicken Burger                     16
Melon Prosciutto                    9
Scones                              3
Name: count, dtype: int64

In [9]:
test_df['itemName'].value_counts()

itemName
Beef Truffle Zappaghetti          116
Beer                               72
Beef Brisket Kimchi Pilaf          62
탄산음료                               49
Chicken Karaage & Fried Dishes     46
에이드                                45
Margherita Pizza                   41
Creamy Pasta                       35
Mini Burger Set                    31
Tteokbokki                         26
Hamburg Steak                      18
Truffle Mushrooms Pizza            18
Pig Hocks Set                      13
Grilled Camembert Cheese           10
Soju                                8
Americano                           8
Abalone Porridge                    6
Chicken Burger                      6
Korean Traditional Pancakes         5
Melon Prosciutto                    4
Name: count, dtype: int64

In [10]:
# 기본적으로 주문이 많이 된 아이템 목록 추출
best_item = train_df['itemName'].value_counts().index.tolist()
print(best_item)

['Beer', 'Chicken Karaage & Fried Dishes', 'Mini Burger Set', '탄산음료', 'Beef Truffle Zappaghetti', '에이드', 'Pig Hocks Set', 'Margherita Pizza', 'Beef Brisket Kimchi Pilaf', 'Soju', 'Americano', 'Prosciutto Rucola Pizza', 'Hamburg Steak', 'Seafood Oil Casareccia Pasta', 'Abalone Porridge', 'Truffle Mushrooms Pizza', 'Tteokbokki', 'Creamy Pasta', 'LA Galbi Set', 'Korean Traditional Pancakes', 'Grilled Camembert Cheese', 'Chicken Burger', 'Melon Prosciutto', 'Scones']


In [11]:
# Stratified K-Fold 적용
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1212)

for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df,train_df['room_name'])):
    print(f"Fold {fold + 1}")

    train_data = train_df.iloc[train_idx].copy()
    valid_data = train_df.iloc[valid_idx].copy()

    train_data.to_csv(f"train_data_{fold}.csv", index=False)
    valid_data.to_csv(f"valid_data_{fold}.csv", index=False)

Fold 1
Fold 2
Fold 3
Fold 4


c:\Users\uiuiu\mlflow_practice\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


In [ ]:
##### 한개의 train_data를 이용하여 latent_cf 를 생성하고 추론 결과를 반환해보기
# 주문 횟수를 평점행렬로 만들어주기
# train_df["count"]=1

# 주문 횟수를 평점행렬로 만들어주기
# ratings_matrix = pd.pivot_table(train_df,values='count',index='room_name',columns='itemName',aggfunc='sum',fill_value=0)
# ratings_matrix


itemName,Abalone Porridge,Americano,Beef Brisket Kimchi Pilaf,Beef Truffle Zappaghetti,Beer,Chicken Burger,Chicken Karaage & Fried Dishes,Creamy Pasta,Grilled Camembert Cheese,Hamburg Steak,...,Mini Burger Set,Pig Hocks Set,Prosciutto Rucola Pizza,Scones,Seafood Oil Casareccia Pasta,Soju,Truffle Mushrooms Pizza,Tteokbokki,에이드,탄산음료
room_name,,,,,,,,,,,,,,,,,,,,,
901,0,0,0,0,0,0,1,0,0,0,...,1,1,0,0,0,1,0,0,0,1
902,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
903,0,1,1,1,3,2,2,0,0,1,...,1,3,0,0,0,2,1,0,2,4
904,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,1,0,0,0,1,8
905,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,1,2,2,1,0,0,1,0,0,1,...,2,1,1,0,0,0,1,0,1,1
2704,0,0,0,0,2,0,1,0,0,0,...,3,0,2,0,0,0,0,0,4,0
2705,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,2,0


In [13]:
##### 한개의 train_data를 이용하여 latent_cf 를 생성하고 추론 결과를 반환해보기
# 주문 횟수를 평점행렬로 만들어주기
ratings_matrix = pd.pivot_table(train_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)
ratings_matrix

itemName,Abalone Porridge,Americano,Beef Brisket Kimchi Pilaf,Beef Truffle Zappaghetti,Beer,Chicken Burger,Chicken Karaage & Fried Dishes,Creamy Pasta,Grilled Camembert Cheese,Hamburg Steak,...,Mini Burger Set,Pig Hocks Set,Prosciutto Rucola Pizza,Scones,Seafood Oil Casareccia Pasta,Soju,Truffle Mushrooms Pizza,Tteokbokki,에이드,탄산음료
room_name,,,,,,,,,,,,,,,,,,,,,
901,0,0,0,0,0,0,1,0,0,0,...,1,1,0,0,0,1,0,0,0,1
902,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
903,0,1,1,1,3,2,2,0,0,1,...,1,3,0,0,0,2,1,0,2,4
904,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,1,0,0,0,1,8
905,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,1,2,2,1,0,0,1,0,0,1,...,2,1,1,0,0,0,1,0,1,1
2704,0,0,0,0,2,0,1,0,0,0,...,3,0,2,0,0,0,0,0,4,0
2705,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,2,0


In [14]:
def get_rmse(R, P ,Q):
    pred_matrix = np.dot(P,Q.T)

    R_not_nans = R[~np.isnan(R)]

    pred_matrix_not_nanas = pred_matrix[~np.isnan(R)]

    rmse = np.sqrt(mean_squared_error(R_not_nans,pred_matrix_not_nanas))

    return rmse

In [29]:
def train_latent_cf(train_matrix,k_factor,learning_rate,r_lambda):
    R = train_matrix
    
    num_rooms, num_items = R.shape

    K = k_factor

    P = np.random.normal(scale=1./K, size=(num_rooms,K))
    Q = np.random.normal(scale=1./K, size=(num_items,K))

    epochs = 1000

    not_nans = [ (i,j,R[i,j]) for i in range(num_rooms) for j in range(num_items) if R[i,j]>0]

    not_nans_index = np.argwhere(R > 0)  # 조건을 만족하는 (i, j) 인덱스 반환
    not_nans_values = R[R > 0]              # 조건을 만족하는 값 반환
    not_nans = [(i, j, val) for (i, j), val in zip(not_nans_index, not_nans_values)]
    
    for epoch in range(epochs):
        for i,j,r in not_nans:
            error_i_j = r - np.dot(P[i,:],Q[j,:].T)
            
            P[i,:] = P[i,:] + learning_rate *(error_i_j*Q[j,:]-r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate *(error_i_j*P[i,:]-r_lambda*Q[j,:])
            
        rmse = get_rmse(R,P,Q)
        if (epoch%100) == 0:
            print("iteration step :",epoch," rmse :",rmse)

    return np.dot(P,Q.T)

In [30]:
class LatentFactorCF(mlflow.pyfunc.PythonModel):
    def __init__(self,pred_df,basic_recommend_item):
        self.pred_df = pred_df
        self.basic_recommend_item = basic_recommend_item
        
    def predict(self, context, model_input, params=None):
        answer = {}
        for room in model_input['room_name']:
            if room in self.pred_df.index:
                target_row = self.pred_df.loc[room]
                sorted_items = target_row.sort_values(ascending=False)
                sorted_item_names = sorted_items.index.tolist()
                answer[room]=sorted_item_names
            else:
                answer[room]=self.basic_recommend_item
        return answer

In [31]:
def calculate_best_item_accuracy(valid_pivot_table, best_items):
    # 모든 호실의 추천된 아이템 구매 횟수 총합
    total_recommended_purchases = 0

    # 전체 구매 횟수
    total_purchases = valid_pivot_table.sum().sum()

    for room_id in valid_pivot_table.index:
        # 상위 5개의 추천 아이템 중 실제 pivot_table에 존재하는 아이템만 필터링
        existing_items = [item for item in best_items[:3] if item in valid_pivot_table.columns]

        if not existing_items:  # 만약 존재하는 아이템이 없다면
            continue  # 해당 room_id는 스킵

        # 추천된 아이템의 구매 횟수 합산
        total_recommended_purchases += valid_pivot_table.loc[room_id, existing_items].sum()

    # 정확도 계산
    accuracy = total_recommended_purchases / total_purchases if total_purchases > 0 else 0
    return accuracy

In [32]:
def calculate_overall_accuracy(valid_pivot_table, recommendations):
    # 모든 호실의 추천된 아이템 구매 횟수 총합
    total_recommended_purchases = 0
    
    # 전체 구매 횟수
    total_purchases = valid_pivot_table.sum().sum()

    for room_id, recommended_items in recommendations.items():
        # 추천된 아이템의 구매 횟수 합산
        # 추천된 아이템 중 실제 valid_pivot_table에 존재하는 아이템만 필터링
        existing_items = [item for item in recommended_items[:3] if item in valid_pivot_table.columns]

        if not existing_items:  # 만약 존재하는 아이템이 없다면
            continue  # 해당 room_id는 스킵

        # 추천된 아이템의 구매 횟수 합산
        total_recommended_purchases += valid_pivot_table.loc[room_id, existing_items].sum()
    
    # 정확도 계산
    accuracy = total_recommended_purchases / total_purchases if total_purchases > 0 else 0
    return accuracy

In [ ]:
mlflow.set_tracking_uri("http://<PublicIP>:5000")
mlflow.set_experiment("hotel_item")

UNIQUE_PREFIX = str(uuid.uuid4())[:8]

# optuna를 통한 HPO 진행
def objective(trial):
    trial.suggest_int("k_factor",64,1024,step=64)
    trial.suggest_float("learning_rate", 1e-4, 5e-2)
    trial.suggest_float("r_lambda", 1e-4,5e-2)

    # n_epoch 동안 모델 학습  
    
    # 모델 학습을 진행하고 해당 결과를 run_name으로 기록하기
    run_name = f"{UNIQUE_PREFIX}-{trial.number}"
    
    with mlflow.start_run(run_name=run_name):
        mlflow.log_params(trial.params)

        latent_accuracy_list = []
        simple_accuracy_list = []
        for fold_idx in range(4):   # 4-Fold
        # 주문 횟수를 평점행렬로 만들어주기
            train_df = pd.read_csv(f"train_data_{fold_idx}.csv")
            valid_df = pd.read_csv(f"valid_data_{fold_idx}.csv")

            # 주문 횟수를 평점행렬로 만들어주기
            ratings_matrix = pd.pivot_table(train_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)

            pred_R = train_latent_cf(ratings_matrix.to_numpy(),k_factor=trial.params["k_factor"],learning_rate=trial.params['learning_rate'],r_lambda=trial.params['r_lambda'])

            pred_df = pd.DataFrame(pred_R,index=ratings_matrix.index,columns=ratings_matrix.columns)
            model = LatentFactorCF(pred_df,best_item)
            
            valid_predict_dict = model.predict(None,valid_df)

            vaild_answer_pivot_table = pd.pivot_table(valid_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)
            
            valid_accuracy = calculate_overall_accuracy(vaild_answer_pivot_table, valid_predict_dict)
            
            
            accuracy = calculate_best_item_accuracy(vaild_answer_pivot_table, best_item)
            
            latent_accuracy_list.append(valid_accuracy)
            simple_accuracy_list.append(accuracy)
            
        mlflow.log_metric("latentcf_recommend_accuracy", np.mean(latent_accuracy_list))
        mlflow.log_metric("simple_recommend_accuracy", np.mean(simple_accuracy_list))

    return np.mean(latent_accuracy_list)


2024/11/20 23:31:27 INFO mlflow.tracking.fluent: Experiment with name 'hotel_item' does not exist. Creating a new experiment.


In [39]:
import optuna

sampler = optuna.samplers.RandomSampler(seed=2024)
study = optuna.create_study(sampler=sampler, study_name="latent_cf", direction="maximize")

# optimize
study.optimize(objective, n_trials=100)


[I 2024-11-20 23:31:28,383] A new study created in memory with name: latent_cf


iteration step : 0  rmse : 0.7926662283274025
iteration step : 100  rmse : 0.948712450796807
iteration step : 200  rmse : 0.9310223310583879
iteration step : 300  rmse : 0.9185616233383278
iteration step : 400  rmse : 0.908862976691379
iteration step : 500  rmse : 0.9010022919719256
iteration step : 600  rmse : 0.8944513476196452
iteration step : 700  rmse : 0.8888902594831242
iteration step : 800  rmse : 0.8841099442099601
iteration step : 900  rmse : 0.8799630287624076
iteration step : 0  rmse : 0.7984930860786991
iteration step : 100  rmse : 0.9555367266503468
iteration step : 200  rmse : 0.9380776037164028
iteration step : 300  rmse : 0.9252192664829255
iteration step : 400  rmse : 0.9153378069568128
iteration step : 500  rmse : 0.9073444376268062
iteration step : 600  rmse : 0.900651095720755
iteration step : 700  rmse : 0.894930828367262
iteration step : 800  rmse : 0.8899786776393707
iteration step : 900  rmse : 0.8856529930475934
iteration step : 0  rmse : 0.7999061014900499
it

2024/11/20 23:32:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run a781fdce-0 at: http://34.64.239.194:5000/#/experiments/688275472755316948/runs/5175a1bc2d5b42af9d0e85aacc513b60.
2024/11/20 23:32:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://34.64.239.194:5000/#/experiments/688275472755316948.
[I 2024-11-20 23:32:35,059] Trial 0 finished with value: 0.2877880075913233 and parameters: {'k_factor': 640, 'learning_rate': 0.03498552650931097, 'r_lambda': 0.009488782805921449}. Best is trial 0 with value: 0.2877880075913233.


iteration step : 0  rmse : 0.7926951189597942
iteration step : 100  rmse : 0.9331011022389968
iteration step : 200  rmse : 0.9300543462669073
iteration step : 300  rmse : 0.9246917072342735
iteration step : 400  rmse : 0.9200101367674178
iteration step : 500  rmse : 0.9161199406314319
iteration step : 600  rmse : 0.9127746049160874
iteration step : 700  rmse : 0.909813604859202
iteration step : 800  rmse : 0.9071453022561625
iteration step : 900  rmse : 0.9047128230212346
iteration step : 0  rmse : 0.7985729989780618
iteration step : 100  rmse : 0.9320312635972194
iteration step : 200  rmse : 0.9325064200181377
iteration step : 300  rmse : 0.928890504932718
iteration step : 400  rmse : 0.9253835071939843
iteration step : 500  rmse : 0.9221277207064211
iteration step : 600  rmse : 0.919117711152914
iteration step : 700  rmse : 0.9163564350889967
iteration step : 800  rmse : 0.9138318879217578
iteration step : 900  rmse : 0.9115206226510318
iteration step : 0  rmse : 0.7999353626922261
i

2024/11/20 23:33:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run a781fdce-1 at: http://34.64.239.194:5000/#/experiments/688275472755316948/runs/bd9775e39e5846eeaffa68922f853594.
2024/11/20 23:33:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://34.64.239.194:5000/#/experiments/688275472755316948.
[I 2024-11-20 23:33:26,747] Trial 1 finished with value: 0.28698025314867387 and parameters: {'k_factor': 64, 'learning_rate': 0.01033044572447501, 'r_lambda': 0.00539253743636566}. Best is trial 0 with value: 0.2877880075913233.


...

2024/11/21 01:15:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run a781fdce-97 at: http://34.64.239.194:5000/#/experiments/688275472755316948/runs/f6250cda0f294b54801afe91a2296ae1.
2024/11/21 01:15:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://34.64.239.194:5000/#/experiments/688275472755316948.
[I 2024-11-21 01:15:45,726] Trial 97 finished with value: 0.2861711916600007 and parameters: {'k_factor': 320, 'learning_rate': 0.02081876647629214, 'r_lambda': 0.004167595241439329}. Best is trial 3 with value: 0.30233804392720276.


iteration step : 0  rmse : 0.7921485819839692
iteration step : 100  rmse : 0.9844006609976625
iteration step : 200  rmse : 0.9658777089832272
iteration step : 300  rmse : 0.9536409557687497
iteration step : 400  rmse : 0.9446442675554769
iteration step : 500  rmse : 0.9375342807476705
iteration step : 600  rmse : 0.9316482439887102
iteration step : 700  rmse : 0.9266185818345085
iteration step : 800  rmse : 0.9222214613113683
iteration step : 900  rmse : 0.9183110142078287
iteration step : 0  rmse : 0.7980229601563316
iteration step : 100  rmse : 0.9922823952671614
iteration step : 200  rmse : 0.9737568969076602
iteration step : 300  rmse : 0.9613980152759422
iteration step : 400  rmse : 0.9523289118847902
iteration step : 500  rmse : 0.9452091186451722
iteration step : 600  rmse : 0.9393414193000112
iteration step : 700  rmse : 0.9343369059290757
iteration step : 800  rmse : 0.9299620945579536
iteration step : 900  rmse : 0.9260672873676419
iteration step : 0  rmse : 0.799452797906778

2024/11/21 01:16:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run a781fdce-98 at: http://34.64.239.194:5000/#/experiments/688275472755316948/runs/6bcecef913d2416497eedfba99665431.
2024/11/21 01:16:53 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://34.64.239.194:5000/#/experiments/688275472755316948.
[I 2024-11-21 01:16:53,589] Trial 98 finished with value: 0.28536539778638687 and parameters: {'k_factor': 640, 'learning_rate': 0.04321593253075656, 'r_lambda': 0.0020280443878982844}. Best is trial 3 with value: 0.30233804392720276.


iteration step : 0  rmse : 0.7919777564830263
iteration step : 100  rmse : 0.9042782801685615
iteration step : 200  rmse : 0.878456777480515
iteration step : 300  rmse : 0.8646589179392584
iteration step : 400  rmse : 0.856694127350109
iteration step : 500  rmse : 0.8517328306149773
iteration step : 600  rmse : 0.848450061876447
iteration step : 700  rmse : 0.8461711641209728
iteration step : 800  rmse : 0.8445276399799957
iteration step : 900  rmse : 0.8433060981947784
iteration step : 0  rmse : 0.797805986025662
iteration step : 100  rmse : 0.9121095914716119
iteration step : 200  rmse : 0.8847919093096132
iteration step : 300  rmse : 0.869966962212407
iteration step : 400  rmse : 0.8613907694681957
iteration step : 500  rmse : 0.8561274188041234
iteration step : 600  rmse : 0.8527464728984875
iteration step : 700  rmse : 0.8504946615514731
iteration step : 800  rmse : 0.8489504205185057
iteration step : 900  rmse : 0.8478658537188699
iteration step : 0  rmse : 0.7993382696027139
ite

2024/11/21 01:18:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run a781fdce-99 at: http://34.64.239.194:5000/#/experiments/688275472755316948/runs/90b80ef820734281a4fac61f1b350ed4.
2024/11/21 01:18:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://34.64.239.194:5000/#/experiments/688275472755316948.
[I 2024-11-21 01:18:03,415] Trial 99 finished with value: 0.29021715262637826 and parameters: {'k_factor': 832, 'learning_rate': 0.048428943906603523, 'r_lambda': 0.03962957394625503}. Best is trial 3 with value: 0.30233804392720276.


In [41]:
best_params = study.best_params
print(best_params)

{'k_factor': 512, 'learning_rate': 0.0010534366988360548, 'r_lambda': 0.03765465702670811}


In [ ]:
import mlflow

mlflow.set_tracking_uri("http://<PublicIP>:5000")
mlflow.set_experiment("hotel_item")


# best_params 기준으로 전체 Train 데이터에 대하여 재학습 진행
best_params = study.best_params

# train/test전체 데이터 가져오기 
train_df = pd.read_csv("train_original.csv")
test_df = pd.read_csv("test_original.csv")

# 주문 횟수를 평점행렬로 만들어주기
ratings_matrix = pd.pivot_table(train_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)

pred_R = train_latent_cf(ratings_matrix.to_numpy(),k_factor=best_params["k_factor"],learning_rate=best_params['learning_rate'],r_lambda=best_params['r_lambda'])

pred_df = pd.DataFrame(pred_R,index=ratings_matrix.index,columns=ratings_matrix.columns)
best_item = ['Beer', 'Chicken Karaage & Fried Dishes', 'Mini Burger Set', '탄산음료', 'Beef Truffle Zappaghetti', '에이드', 'Pig Hocks Set', 'Margherita Pizza', 'Beef Brisket Kimchi Pilaf', 'Soju', 'Americano', 'Prosciutto Rucola Pizza', 'Hamburg Steak', 'Seafood Oil Casareccia Pasta', 'Abalone Porridge', 'Truffle Mushrooms Pizza', 'Tteokbokki', 'Creamy Pasta', 'LA Galbi Set', 'Korean Traditional Pancakes', 'Grilled Camembert Cheese', 'Chicken Burger', 'Melon Prosciutto', 'Scones']

model = LatentFactorCF(pred_df,best_item) 

# 학습된 모델로 test 데이터에 추론 결과 살펴보기
test_predict_dict = model.predict(None,test_df)
test_answer_pivot_table = pd.pivot_table(test_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)

test_accuracy = calculate_overall_accuracy(test_answer_pivot_table, test_predict_dict)
accuracy = calculate_best_item_accuracy(test_answer_pivot_table, best_item)
print("모델 정확도 :",test_accuracy)
print("단순추천 정확도 :",accuracy)


iteration step : 0  rmse : 0.9874741172187775
iteration step : 100  rmse : 0.7543788501372649
iteration step : 200  rmse : 0.9310470478348734
iteration step : 300  rmse : 0.9728428057055286
iteration step : 400  rmse : 0.9652514992747327
iteration step : 500  rmse : 0.9557550691182802
iteration step : 600  rmse : 0.9504357172990253
iteration step : 700  rmse : 0.9497479298903917
iteration step : 800  rmse : 0.9508793922168942
iteration step : 900  rmse : 0.9519968148272206
모델 정확도 : 0.27463651050080773
단순추천 정확도 : 0.2407108239095315


In [ ]:
# 모델 입력 예제를 DataFrame으로 정의
input_example = pd.DataFrame({
    "room_name": [901, 902, 2704],
})

with mlflow.start_run(run_name="Best_Model_Run") as run:
    model_info = mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=model,
        input_example=input_example,
    )

In [51]:
# 모델 registry에 등록
model_uri = f"runs:/{run.info.run_id}/model"
registered_model = mlflow.register_model(model_uri=model_uri, name="latent_best_model") 

Registered model 'latent_best_model' already exists. Creating a new version of this model...
2024/11/21 02:43:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: latent_best_model, version 3
Created version '3' of model 'latent_best_model'.


In [ ]:
import requests
import json

url = "http://<PublicIP>:5001/invocations"
headers = {"Content-Type": "application/json"}

data = {
    "dataframe_split": {
        "columns": ["room_name"],
        "data": [2606,1906]
    }
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

{'predictions': {'2606': ['Beer', '에이드', '탄산음료', 'Beef Truffle Zappaghetti', 'Prosciutto Rucola Pizza', 'Beef Brisket Kimchi Pilaf', 'Abalone Porridge', 'Americano', 'Seafood Oil Casareccia Pasta', 'Chicken Burger', 'LA Galbi Set', 'Pig Hocks Set', 'Truffle Mushrooms Pizza', 'Mini Burger Set', 'Scones', 'Korean Traditional Pancakes', 'Grilled Camembert Cheese', 'Hamburg Steak', 'Creamy Pasta', 'Soju', 'Margherita Pizza', 'Tteokbokki', 'Melon Prosciutto', 'Chicken Karaage & Fried Dishes'], '1906': ['Beer', 'Chicken Karaage & Fried Dishes', '탄산음료', 'Mini Burger Set', 'Pig Hocks Set', '에이드', 'Beef Truffle Zappaghetti', 'Soju', 'Americano', 'Abalone Porridge', 'Margherita Pizza', 'Prosciutto Rucola Pizza', 'Chicken Burger', 'Tteokbokki', 'LA Galbi Set', 'Seafood Oil Casareccia Pasta', 'Hamburg Steak', 'Beef Brisket Kimchi Pilaf', 'Creamy Pasta', 'Truffle Mushrooms Pizza', 'Melon Prosciutto', 'Korean Traditional Pancakes', 'Scones', 'Grilled Camembert Cheese']}}


In [ ]:
import requests
import json

test_df = pd.read_csv("test_original.csv")


url = "http://<publicIP>:5001/invocations"
headers = {"Content-Type": "application/json"}

data = {
    "dataframe_split": test_df.to_dict(orient="split")
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

# 학습된 모델로 test 데이터에 추론 결과 살펴보기
# test_predict_dict = model.predict(None,test_df)
test_predict_dict = requests.post(url, headers=headers, data=json.dumps(data)).json()['predictions']
test_predict_dict = {int(k): v for k, v in test_predict_dict.items()}
test_answer_pivot_table = pd.pivot_table(test_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)

test_accuracy = calculate_overall_accuracy(test_answer_pivot_table, test_predict_dict)
accuracy = calculate_best_item_accuracy(test_answer_pivot_table, best_item)
print("모델 정확도 :",test_accuracy)
print("단순추천 정확도 :",accuracy)

{'predictions': {'2306': ['Mini Burger Set', 'Beer', 'Beef Truffle Zappaghetti', 'Americano', 'Abalone Porridge', 'Margherita Pizza', 'Tteokbokki', 'Beef Brisket Kimchi Pilaf', 'Chicken Karaage & Fried Dishes', 'Korean Traditional Pancakes', '탄산음료', 'Chicken Burger', '에이드', 'Seafood Oil Casareccia Pasta', 'Grilled Camembert Cheese', 'Truffle Mushrooms Pizza', 'Melon Prosciutto', 'Pig Hocks Set', 'Creamy Pasta', 'LA Galbi Set', 'Hamburg Steak', 'Scones', 'Prosciutto Rucola Pizza', 'Soju'], '2507': ['Mini Burger Set', 'Chicken Karaage & Fried Dishes', '에이드', 'Seafood Oil Casareccia Pasta', 'Margherita Pizza', 'Beef Truffle Zappaghetti', 'Abalone Porridge', 'Americano', 'Tteokbokki', 'Truffle Mushrooms Pizza', 'Chicken Burger', 'Korean Traditional Pancakes', 'Pig Hocks Set', 'Creamy Pasta', '탄산음료', 'Melon Prosciutto', 'LA Galbi Set', 'Beer', 'Beef Brisket Kimchi Pilaf', 'Grilled Camembert Cheese', 'Hamburg Steak', 'Scones', 'Prosciutto Rucola Pizza', 'Soju'], '1805': ['탄산음료', '에이드', 'Mini